In [1]:
from datetime import datetime
import functools as ft
import gc
import numpy as np
import pickle
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy import types, create_engine
from dateutil.relativedelta import relativedelta

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

In [2]:
conn_sql = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=S26;Database=Analytics_WS;Trusted_Connection=yes')

driver = '{ODBC Driver 17 for SQL Server}'
server = 'S28'
db = 'Analytics_WS'
engine_sql = create_engine("mssql+pyodbc:///?odbc_connect=" + 'Driver=%s;Server=%s;Database=%s;QuotedID=Yes;AnsiNPW=Yes;Trusted_Connection=yes' % (driver, server, db))

In [3]:
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
engine = create_engine(URL(
    account='idb59911.us-west-2',
    user='NLU@NATIONALFUNDING.COM', # change to your username
    authenticator = 'externalbrowser',
    database='SANDBOX',
    role = 'SANDBOX_NLU', # change to your username
    schema='NLU', # change to your username
))
conn = engine.connect()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://nationalfunding.okta.com/app/snowflake/exk8t3c21w5IwCyzl5d7/sso/saml?SAMLRequest=jVLbctowFPwVj%2FqMJdmhAQ0mwyWZMA0JBUNL3xRLEBVZciU5hn59ZRs66UMyfdNlz9k9Z3dwc8xl8MqNFVolAIcIBFxlmgm1T8A6vev0QGAdVYxKrXgCTtyCm%2BHA0lwWZFS6F7Xkv0puXeAbKUvqjwSURhFNrbBE0Zxb4jKyGs0fSBQiQq3lxnk6cC5hVniuF%2BcKAmFVVWEVh9rsYYQQgqgPPaqGfAJvKIqPOQqjnc60vJQc%2FUzvUGCIrmoKj%2FAMd9pkvBkrAc6UHASLc6uxUO1SPuJ9bkGW3KfporN4WqUgGF3mnWhly5ybFTevIuPr5UMryXpNgj13%2B32MQ6t0tZP0wDOdF6XzzUJ%2FgjvOoNR74Zc2myagOAi2XX5N51m60qtbVsbjb9vlfXf%2Bc3Mq5GF7NcWPu%2B%2BbtRzfxl8YykCwuVgc1RbPrC35TNXGOv%2BEoriDow7qpbhHIkzwdYg%2BRz9AMPXGCkVdU3nR2t6p3JXNsKE%2BONqIpEUB%2F%2BqH%2FHjouTiLcNWdVZPTb9ll19BaDWv%2FQBsh0ggxw%2F9cwwC%2BLTqH8NG7MJsutBTZqfYvp%2B59k3CImxfBOrsGSnhOhRwxZri13iwpdTUxnDp%2BDgActqz%

# Import data

In [5]:
print(pd.__version__)

1.4.4


In [7]:
sql = """
select unique_id, run_date, company, business_dba, address, city, state, zip
from vendor_test_data_internal m
"""
df = pd.read_sql(sql, engine)
df.shape

(20000, 8)

In [8]:
df.head()

,unique_id,run_date,company,business_dba,address,city,state,zip
0,09537,2023-07-01,Ack Transportation Corp.,ACK Transportation,1435 Beach Ave,Bronx,NY,10460
1,11182,2023-06-01,Craftsman Market LLC,Craftsman Cabinets and Furniture,3367 Helton Hartley Pl,Lenoir,NC,28645
2,09893,2023-07-01,"Hilltop Family Restaurant, LLC",Hilltop Family Restaurant,18047 US Highway 41,Lanse,MI,49946
3,15085,2023-03-01,Tab's Grill & Cantina LLC,Tab's Grill & Cantina LLC,11 Main St,Canton,NY,13617
4,14193,2022-12-01,"Brito's Construction, Inc.",Brito's Construction,9062 Sherlock Cir,Garden Grove,CA,92841


In [9]:
df.sort_values(by=['unique_id', 'run_date'], inplace=True)
df.to_csv('vendor_test_data.csv', index=False, header=True)

In [13]:
df2 = pd.read_csv('vendor_test_data.csv')
df2.shape

(20000, 8)

In [14]:
df2.head()

,unique_id,run_date,company,business_dba,address,city,state,zip
0,0,2023-01-01,Loyal Vending & General Services LLC,NaN,12365 SW 255th Ter,Homestead,FL,33032
1,1,2023-04-01,Trust Multiple Services LLC,NaN,3427 Malaga Way,Naples,FL,34105
2,2,2022-06-01,Legacy Logistics 365 LLC,NaN,9478 Garfield Cir,Shreveport,LA,71118
3,3,2022-11-01,"Black Girls Build, LLC",NaN,312 Bucknell Cir,Waldorf,MD,20602
4,4,2021-12-01,Bland Transportation LLC,NaN,2741 N 20th St,Philadelphia,PA,19132


# whole data

In [44]:
sql = """
select distinct dat.*
, customer.business_name
, customer.business_start_date
, customer.sic_code
, customer.state as customer_state
, opportunity.nf_salesforce_opportunity_id
, customer.customer_key
from vendor_test_data_internal dat
left join ods.salesforce_nf.sf_nf_credit_review__c as credit_review
on dat.best_creditreviewid = credit_review.credit_review__c_id
left join datawarehouse.core.opportunity opportunity
on credit_review.opportunity__c=opportunity.nf_salesforce_opportunity_id and opportunity.brand='National Funding'
left join datawarehouse.core.customer as customer 
on opportunity.customer_key = customer.customer_key
"""
df = pd.read_sql(sql, engine)
df.shape

(20000, 27)

In [48]:
df.count()

unique_id                       20000
prospect_type                   20000
accountnum                      19000
campaignid                      20000
lead_key                        16000
leadid                          19000
nf_salesforce_opportunity_id     7606
customer_key                     7606
duns                            20000
run_date                        20000
mail_date                       20000
company                         20000
address                         20000
city                            20000
state                           20000
zip                             19999
flg_type                        20000
flg_type_grp                    20000
business_name                    7606
business_dba                     7929
best_creditreviewid              7606
credit_review_ags                7606
business_start_date              7606
sic_code                         5116
customer_state                   7582
dtype: int64

In [45]:
df = df[['unique_id', 'prospect_type', 'accountnum', 'campaignid'
        , 'lead_key', 'leadid'
        , 'nf_salesforce_opportunity_id','customer_key'
        , 'duns', 'run_date', 'mail_date', 'company'
        , 'address', 'city', 'state', 'zip'
        , 'flg_type', 'flg_type_grp'
        , 'business_name', 'business_dba'
        , 'best_creditreviewid', 'credit_review_ags'
        ,  'business_start_date', 'sic_code', 'customer_state']].copy()

In [46]:
df.sort_values(by=['unique_id', 'run_date'], inplace=True)
df.to_csv('vendor_test_data_internal.csv', index=False, header=True)